## Implementation of Lagrangian Relaxation Method for CLSP

###### Relaxing the constraint which restrict the production capacity for each machine j at each period t, by introducing penalties in the objective function.

In [1]:
from docplex.mp.model import Model

In [2]:
def Lagrangian_relaxation(Costs,Vars,no_of_items,no_of_machines,no_of_periods, max_iters=101):
    """
    Function to solve CLSP after relaxing constraint_2
    Parameter: 
                Costs: List of varied Costs, 
                Vars:  List of x_variables,
    Returns: CPLEX SolveSolution type object
    """
    with Model('CLSP -with- Lagrangian Relaxation') as mdl:
        
        Item_list = [i for i in range(1,no_of_items+1)]
        Machine_list = [j for j in range(1,no_of_machines+1)]
        Period_list = [t for t in range(1,no_of_periods+1)]

        item_period_pair = [(i,t) for i in Item_list for t in Period_list]
        machine_period_pair = [(j,t) for j in Machine_list for t in Period_list]
        
        # Assgining costs
        Bij = Costs[0]
        Cij = Costs[1]
        Hi = Costs[2]
        Dit = Costs[3]
        Fij = Costs[4]
        Sij = Costs[5]
        Qj = Costs[6]
        M = Costs[7]
        
        # variables
        X_vars = mdl.integer_var_dict(Vars, lb=0, name='x')
        Y_vars = mdl.binary_var_dict(Vars,name='y')
        S_vars = mdl.integer_var_dict(item_period_pair, lb=0, name='s')
        P_vars = mdl.continuous_var_dict(machine_period_pair, name='p')                     # Introducing the penalty variables

        same_constraint_1 = mdl.add_constraints(mdl.sum(X_vars[i,j,t] for j in range(1,no_of_machines+1)) + (-S_vars[i,t] + S_vars[i,t-1] if t-1 > 0 else -S_vars[i,t]) == Dit[i,t] for i,t in S_vars)
        updated_constraint_2 = mdl.add_constraints(mdl.sum(Bij[i,j]*X_vars[i,j,t] + Fij[i,j]*Y_vars[i,j,t] for i in range(1,no_of_items+1)) == Qj[j] - P_vars[j,t] for j,t in zip(Machine_list,Period_list))
        same_constraint_3 = mdl.add_constraints(X_vars[i,j,t] <= M*Y_vars[i,j,t] for i,j,t in X_vars)

        # lagrangian relaxation loop
        epsilon = 1e-6
        loop_count = 0
        best_solution = 0
        initial_multiplier = 1
        multipliers = [initial_multiplier] * (no_of_machines*no_of_periods)
        
        Setup_Cost = mdl.sum(Sij[i,j]*Y_vars[i,j,t] for i,j,t in Y_vars) 
        Production_Cost = mdl.sum(Cij[i,j]*X_vars[i,j,t] for i,j,t in X_vars)
        Inventory_Cost = mdl.sum(Hi[i]*S_vars[i,t] for i,t in S_vars)
        total_cost = Setup_Cost + Production_Cost + Inventory_Cost

        mdl.add_kpi(total_cost, "Total Cost")

        while loop_count <= max_iters:
            loop_count += 1
            # rebuilt at each loop iteration
            total_penalty = mdl.sum(multipliers[j]*P_vars[j,t] for j,t in P_vars)
            mdl.minimize(total_cost + total_penalty)
            solution = mdl.solve()
            if not solution:
                print("*** solve fails, stopping at iteration: %d" % loop_count)
                break
            best_solution = solution.objective_value
            pen = solution.get_value_dict(P_vars)
            penalties = [pen[i] for i in pen]
            print('%d> new lagrangian iteration:\n\t obj=%g, m=%s, p=%s' % (loop_count, best_solution, str(multipliers), str(penalties)))

            do_stop = True
            justifier = 0
            for k in range(no_of_machines*no_of_periods):
                penalized_violation = penalties[k] * multipliers[k]
                if penalized_violation >= epsilon:
                    do_stop = False
                    justifier = penalized_violation
                    break

            if do_stop:
                print("* Lagrangian relaxation succeeds, best_solution={:g}, penalty={:g}, #iterations={}".format(best_solution, total_penalty.solution_value, loop_count))
                break
            else:
                # update multipliers and start loop again.
                scale_factor = 1.0 / float(loop_count)
                multipliers = [max(multipliers[i] - scale_factor * penalties[i], 0.) for i in range(no_of_machines*no_of_periods)]
                print('{0}> -- loop continues, m={1!s}, justifier={2:g}'.format(loop_count, multipliers, justifier))

    return solution